In [ ]:
"""
Processes the raw bike trip data to get information on bike dock locations
and when the docks were placed in those locations.

Desired output columns:

id | first | last | name | lat | lon | rides

where
- id is the station's id
- first is the earliest trip date for the station id
- last is the latest trip date for the station id (included in case docks are removed)
- name is the station's name
- lat and lon are the latitude and longitude of the station's location
- rides is a count of the number of rides found in the data -- it is used to remove dummy stations in the data.
    only stations with more than RIDES_COUNT_THRESHOLD are included in output

This script is abstracte to apply to multiple cities.
DON'T FORGET: update the 'CITY' variable

"""
from datetime import datetime
import math
import os

import pandas as pd
from zipfile import ZipFile
CITY = 'dc'
# CITY = 'boston'
# CITY = 'nyc'
print('city', CITY)


RIDES_COUNT_THRESHOLD = 100


def get_filepath(city):
    return '../data/' + city + '-bike/'


def transform_date(date):
    try:
        dt = datetime.strptime(date.split(' ')[0], '%m/%d/%Y')
    except ValueError:
        # this dataset is so frustrating lol
        dt = datetime.strptime(date.split(' ')[0], '%Y-%m-%d')
        
    return dt.strftime('%Y-%m-%d')

def open_zipfile(zipfilename):
    # Because someone dropped some gnarly mac osx files into their zips
    zipfile = ZipFile(zipfilename)
    filenames = [f.filename for f in zipfile.infolist()]
    # Return the first file that can be opened  - not all of them have .csv suffix
    for filename in filenames:
        try:
            df = pd.read_csv(zipfile.open(filename))
            return df
        except:
            print('failed to open filename from zip', zipfilename, ': ', filename)
            pass
    raise Exception('unable to read a csv from zipfile %s' % zipfilename)

def open_zipfile_dc(zipfilename):
## DC bike files from 2012 to 2017 have 4 files for each quarter
## this generator yields each of those files

    zipfile = ZipFile(zipfilename)
    files = [f.filename for f in zipfile.infolist()]

    for filename in files:
        try:
            df = pd.read_csv(zipfile.open(filename))
            yield df
        except:
            continue

In [ ]:
"""
make a dict like 
{"id": {"name": "", "lat": "", "lon": "", "first": "", "last": ""}}
where there is one entry for each id
and where the start time is always the earliest found

and then later transform it into a dict like

{'id': [id1, id2, id3], 'col_2': ['a', 'b', 'c', 'd']}

to then make into a dataframe and save as a CSV
"""

# input file column names for indexing data with
start_station_id = 'startstationid'
start_station_name = 'startstationname'
start_station_latitude = 'startstationlatitude'
start_station_longitude = 'startstationlongitude'
starttime = 'starttime'


    
# output file column names
ID = 'id'
NAME = 'name'
LAT = 'lat'
LON = 'lon'
FIRST = 'first'
LAST = 'last'
RIDES = 'rides'


In [ ]:

def preprocess_stations_df(df):
    # Because someone can't make data files with uniform column names
    df.columns = df.columns.str.replace('number', 'id')  # 'Station Number' vs Station ID
    df.columns = df.columns.str.replace('date', 'time')  # 'Start Date' vs 'Start Time'
    
    
    df.columns = map(str.lower, df.columns)
    df.columns = df.columns.str.replace('[\ ]', '')
    
    # transform the dates
    df[starttime] = df[starttime].apply(transform_date)
    if CITY == "boston":
        df = preprocess_boston_stations_df(df)
    return df


# Some of the earlier bostons stations data does not include lat,lon coordinates.
# These files contains the lat,lon coordinates (and other data) for station IDs
hubway_stations_locations_filenames = [
    "Hubway_Stations_as_of_July_2017.csv",
    "previous_Hubway_Stations_as_of_July_2017.csv"
]

def get_hubway_stations_locations_df():
    df = pd.DataFrame()
    filenames = [get_filepath(CITY) + fname for fname in hubway_stations_locations_filenames]
    for filename in filenames:
        new_df = pd.read_csv(filename)    
        hubway_stations_locations_column_names = {
            "Station ID": start_station_id,
            "Latitude": start_station_latitude,
            "Longitude": start_station_longitude,
        }
        # Rename the column names to match the rides data that the locations data will be joined with
        new_df.rename(columns=hubway_stations_locations_column_names, inplace=True)
        df = new_df if df.empty else df.append(new_df)
    df.drop_duplicates(subset=[start_station_id], inplace=True)
    return df

hubway_stations_locations_df = None
if CITY == "boston":
    hubway_stations_locations_df = get_hubway_stations_locations_df()


def preprocess_boston_stations_df(df):
    if start_station_latitude in df.columns:
        return df
    # Otherwise this is one of the datasets that is lacking lat, lon info.
    # Add the lat,lon info
    return hubway_stations_locations_df.merge(df, on=start_station_id)



# hubway_stations_locations_df.head()

In [10]:
FILENAMES_TO_IGNORE = hubway_stations_locations_filenames + ['stations.csv'] + ['stations.json'] # + [more bad filenames here]


def stations_dict_to_df(stations_dict):
    new_dict = {
        ID: [],
        NAME: [],
        LAT: [],
        LON: [],
        FIRST: [],
        LAST: [],
        RIDES: []
    }
    for station_id, station_dict in stations_dict.items():
        new_dict[ID].append(station_id)
        new_dict[NAME].append(station_dict[NAME])
        new_dict[LAT].append(station_dict[LAT])
        new_dict[LON].append(station_dict[LON])
        new_dict[FIRST].append(station_dict[FIRST])
        new_dict[LAST].append(station_dict[LAST])
        new_dict[RIDES].append(station_dict[RIDES])
    
    return pd.DataFrame.from_dict(new_dict)
    


stations_dict = dict()
needs_lat_lon = set()
directory = get_filepath(CITY)
files_count = 0

for filename in os.listdir(directory):
    print(filename)
    if filename in FILENAMES_TO_IGNORE:# or filename[:6] not in ['202102']:
        continue
    if filename[:4] in ['2021', '2022'] and filename[:6] not in ['202101'] and CITY == 'boston' or (filename[:4] in ['2020', '2021', '2022'] and filename[:6] not in ['202001', '202002', '202003']) and CITY == 'dc' :
        start_station_id = 'start_station_id'
        start_station_name = 'start_station_name'
        start_station_latitude = 'start_lat'
        start_station_longitude = 'start_lng'
        starttime = 'started_at'
    elif CITY != 'dc':
        start_station_id = 'startstationid'
        start_station_name = 'startstationname'
        start_station_latitude = 'startstationlatitude'
        start_station_longitude = 'startstationlongitude'
        starttime = 'starttime'
    else:
        start_station_id = 'startstationid'
        start_station_name = 'startstation'
        start_station_latitude = ""
        start_station_longitude = ""
        starttime = "starttime"


    fullfilename = directory + filename
    print(files_count, ': handling file', filename)
    files_count+=1
    
    if filename.endswith(".csv"):
        stations_dfs = [pd.read_csv(fullfilename)]
    elif filename.endswith(".zip") and CITY != 'dc':
        stations_dfs = [open_zipfile(fullfilename)]
    elif filename.endswith(".zip") and CITY == 'dc':
        stations_dfs = [df for df in open_zipfile_dc(fullfilename)]
    else:
        continue
    
    for stations_df in stations_dfs:

        stations_df = preprocess_stations_df(stations_df)
        
        unique_station_ids = stations_df[start_station_id].unique()
        for station_id in unique_station_ids:
            station_df = stations_df[stations_df[start_station_id] == station_id]
            
            if station_id not in stations_dict:
                try:
                    stations_dict[station_id] = {
                        NAME: station_df[start_station_name].iloc[0], 
                        LAT: station_df[start_station_latitude].iloc[0],
                        LON: station_df[start_station_longitude].iloc[0], 
                        FIRST: station_df[starttime].iloc[0], 
                        LAST: station_df[starttime].iloc[0],
                        RIDES: 0,
                    }
                except Exception as e:
                    
                    if type(e).__name__ == 'KeyError':
                        
                        stations_dict[station_id] = {
                            NAME: station_df[start_station_name].iloc[0], 
                            LAT: 0,
                            LON: 0,
                            FIRST: station_df[starttime].iloc[0], 
                            LAST: station_df[starttime].iloc[0],
                            RIDES: 0,
                        }
                        needs_lat_lon.add(station_id)
                    else:    
                        continue
            if station_id in needs_lat_lon and starttime == 'started_at':
                stations_dict[station_id][LAT] = station_df[start_station_latitude].iloc[0]
                stations_dict[station_id][LON] = station_df[start_station_longitude].iloc[0]
                needs_lat_lon.remove(station_id)
            rides_count = len(station_df.index)
            stations_dict[station_id][RIDES] += rides_count
            station_df = station_df.sort_values(by=[starttime])
            if (station_df[starttime].iloc[0] < stations_dict[station_id][FIRST]):
                stations_dict[station_id][FIRST] = stations_df[starttime].iloc[0]
            if (station_df[starttime].iloc[-1] > stations_dict[station_id][LAST]):
                stations_dict[station_id][LAST] = stations_df[starttime].iloc[-1]


stations_df = stations_dict_to_df(stations_dict)
stations_df.head()

2010-capitalbikeshare-tripdata.zip
0 : handling file 2010-capitalbikeshare-tripdata.zip


C:\Users\Walter\AppData\Local\Temp\ipykernel_25420\344671069.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('[\ ]', '')


2011-capitalbikeshare-tripdata.zip
1 : handling file 2011-capitalbikeshare-tripdata.zip


C:\Users\Walter\AppData\Local\Temp\ipykernel_25420\344671069.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('[\ ]', '')


2012-capitalbikeshare-tripdata.zip
2 : handling file 2012-capitalbikeshare-tripdata.zip


...

C:\Users\Walter\AppData\Local\Temp\ipykernel_25420\344671069.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('[\ ]', '')


202205-capitalbikeshare-tripdata.zip
60 : handling file 202205-capitalbikeshare-tripdata.zip


C:\Users\Walter\AppData\Local\Temp\ipykernel_25420\344671069.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('[\ ]', '')


stations.csv
stations.json


,id,name,lat,lon,first,last,rides
0,31208,M St & New Jersey Ave SE,38.87630,-77.003700,2010-09-20,2022-05-29,96870
1,31209,1st & N St SE,38.87430,-77.005700,2010-09-20,2022-05-29,71502
2,31600,5th & K St NW,38.90304,-77.019027,2010-09-20,2022-05-29,271413
3,31100,19th St & Pennsylvania Ave NW,38.90030,-77.042900,2010-09-20,2022-05-29,106021
4,31109,7th & T St NW,38.91550,-77.022200,2010-09-20,2022-05-29,192171


In [11]:
# Transform the stations_df

# Remove dummy stations (there are test stations in the data)
# Remove stations with less than RIDES_COUNT_THRESHOLD rides
bad_stations_df = stations_df[stations_df[RIDES] < RIDES_COUNT_THRESHOLD]
print('removing %d bad stations that each have less than %d rides from stations data' % (bad_stations_df.shape[0], RIDES_COUNT_THRESHOLD))
stations_df = stations_df[stations_df[RIDES] >= RIDES_COUNT_THRESHOLD]
## Remove stations that do not have a latitude and longitude measure
bad_stations_2 = stations_df[stations_df[LAT] == 0]
print(f'removing {bad_stations_2.shape[0]} more stations for not having latitude or longitude')
stations_df = stations_df[stations_df[LAT] != 0]

removing 405 bad stations that each have less than 100 rides from stations data
removing 4 more stations for not having latitude or longitude


In [ ]:
bad_stations_df.head(10)

In [ ]:
"""For the boston hubway/blue bikes data there will be duplicates because
when management changed from hubway to Bluebikes, the data fromat did too
This includes the station id/numbers and names AND lat/lon!
Task: deduplicate stations

Idea to understand data: sort the stations so the potential duplicates are next to each other
when merging/deduping data make sure to keep the earliest first and the latest last.

approach to deduplicating stations:
- normalize names and add new temporary column with normalized name
- get list of unique normalized names
- for each name:
    make a df for that name, sorted by [first, last]
    update main df to replace entries with that name with:
        first first
        last last
        last name
        rides as sum of rides
    sort main df by [name, first] and drop duplicates (duplicates on normalized name)
    remove normalized name column
"""

import re

NORMALIZED_NAME = 'normalized_name'

def normalized_station_name(name):
    normalized_name = name.lower()
    normalized_name = normalized_name.replace("former",  "").replace(" ", "")
    normalized_name = re.sub(r'[^a-z0-9]','', normalized_name)
    return normalized_name

if CITY == 'boston':
    stations_df[NORMALIZED_NAME] = stations_df[NAME].apply(normalized_station_name)
    normalized_names = stations_df[NORMALIZED_NAME]
    print(normalized_names.shape[0], ' names')
    unique_normalized_names = stations_df[NORMALIZED_NAME].unique()
    print(unique_normalized_names.shape[0], ' unique normalized names') #, unique_normalized_names)


    n = 0
    for normalized_name in unique_normalized_names:
        print(n, 'handling name', normalized_name)
        n+=1
        name_df = stations_df[stations_df[NORMALIZED_NAME] == normalized_name]
        name_df.sort_values(by=[FIRST, LAST], inplace=True)
        first = name_df[FIRST].iloc[0]
        last = name_df[LAST].iloc[-1]
        name = name_df[NAME].iloc[-1]
        rides = name_df[RIDES].sum()
        update_condition = (stations_df[NORMALIZED_NAME] == normalized_name)
        stations_df.loc[update_condition, [FIRST, LAST, NAME, RIDES]] = first, last, name, rides

    stations_dropped_duplicates_df = stations_df.drop_duplicates(subset=[NORMALIZED_NAME])
    print('dropped %s rows based on duplicate names' % (int(stations_df.shape[0]) - int(stations_dropped_duplicates_df.shape[0])))
    stations_dropped_duplicates_df.drop(labels=[NORMALIZED_NAME], axis=1, inplace=True)
    
    stations_df = stations_dropped_duplicates_df


In [ ]:
stations_df = stations_dropped_duplicates_df
stations_df.head(30)

In [12]:
# Save the data to CSV
save_to_csvfilename = directory + 'stations.csv'
stations_df.to_csv(save_to_csvfilename)
print('wrote data to ', save_to_csvfilename)

wrote data to  ../data/dc-bike/stations.csv


In [13]:
# Save the data to JSON that will be used in web app
import json

stations = []
for index, row in stations_df.iterrows():
    # Transform the date
    date = row[5]
    
    stations.append({
        ID: str(row[ID]),
        NAME: row[NAME],
        LAT: row[LAT],
        LON: row[LON],
        FIRST: transform_date(row[FIRST]),
        LAST: transform_date(row[LAST]),
    })

json = json.dumps(stations)

save_to_jsonfilename = directory + 'stations.json'
with open(save_to_jsonfilename, 'w') as f:
    f.write(json)
print("Data written to stations.json")

Data written to stations.json


In [14]:
stations_df.head()

,id,name,lat,lon,first,last,rides
0,31208,M St & New Jersey Ave SE,38.87630,-77.003700,2010-09-20,2022-05-29,96870
1,31209,1st & N St SE,38.87430,-77.005700,2010-09-20,2022-05-29,71502
2,31600,5th & K St NW,38.90304,-77.019027,2010-09-20,2022-05-29,271413
3,31100,19th St & Pennsylvania Ave NW,38.90030,-77.042900,2010-09-20,2022-05-29,106021
4,31109,7th & T St NW,38.91550,-77.022200,2010-09-20,2022-05-29,192171
